5.1  Register for a stream of Twitter data

5.2  Create a bloom filter classifying two days worth of twitters  ( after removing stop words and urls )

5.3  For another days worth of twitter data find the previous twitters that match in the bloom filter
(This means get two days of data in one file or directory , use that data for training the bloom filter, capture a different days data in a different file ( or do it in real time)and capture the match output then running the new twitter data through the filter.

5.4 Plot a historgram of matches for each twitter in 5.3

For the 4-5 grade.- Submit in a separate notebook - YourNAME-Homework5-Supplement

1. Use a different machine learning training algorithm
2. Make a continous feed where you take two days of data and match the incoming stream ( do this for 5 days windowing the filter data)
3. Find new trends in the twitter feed (daily or hourly)
4. Or some other streaming exploration of your choosing

In [9]:
%%writefile code/twitter_streaming_sports_DV.py
# %load code/twitter_streaming_sports_DV.py


#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream

#Variables that contains the user credentials to access Twitter API
access_token = "131711504-uteEX6y8zoi0v4miLzEjZefg22ZZ8TgukyU9bHYW"
access_token_secret = "UH3dk5s00euTgzJ92xveSSi7b84Aw2rrrtFeHelV5rmIv"
consumer_key = "7lKe94q4mhbeINgp1hFWmpdGj"
consumer_secret = "T46JL4SAhMK3EmiXpCFzSVwQ6ipvQ0xfqHBGbAUbPBVjtbDDp1"


#This is a basic listener that just prints received tweets to stdout.
class StdOutListener(StreamListener):        
    def on_data(self, data):
        print(data)
        return True

    def on_error(self, status):
        print(status)


if __name__ == '__main__':

    #This handles Twitter authetification and the connection to Twitter Streaming API
    l = StdOutListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    stream = Stream(auth, l)

    #This line filter Twitter Streams to capture data by the keywords: 'cricket', 'soccer', 'basketball', 'baseball', 'tennis'
    stream.filter(track=['cricket', 'soccer', 'basketball', 'baseball', 'tennis'])

Overwriting code/twitter_streaming_sports_DV.py


In [ ]:
%%sh  
python code/Twitter_streaming_sports_DV.py > tweepy_sports_test1.txt

#### Clean the train data

In [1]:
%%writefile code/twitter_clean_train_DV.py

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\D+')

with open('data/tweepy_sports_train.txt') as f:
    for word in f:
        if word.strip()!='' and len(word.split())>=3:
            word_list=tokenizer.tokenize(word.lower().replace('"text":','').replace('\\u','').replace('https','').replace('\\','').replace('//','')\
                                         .replace('/','').replace('\\','').replace('"','').split(',')[3])
           
            for word_ in word_list:
                if word_ not in stopwords.words('english') and not(word_.startswith('u30')) and len(word_.strip())>2:
                    print(word_.split())


Writing code/twitter_clean_train_DV.py


In [2]:
%%sh  
python code/twitter_clean_train_DV.py > data/tweepy_cleaned_train.txt

bash.exe: warning: could not find /tmp, please create!


#### Clean the test data

In [3]:
%%writefile code/twitter_clean_test_DV.py

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\D+')

with open('data/tweepy_sports_test.txt') as f:
    for word in f:
        if word.strip()!='' and len(word.split())>=3:
            word_list=tokenizer.tokenize(word.lower().replace('"text":','').replace('\\u','').replace('https','').replace('\\','').replace('//','')\
                                         .replace('/','').replace('\\','').replace('"','').split(',')[3])
           
            for word_ in word_list:
                if word_ not in stopwords.words('english') and not(word_.startswith('u30')) and len(word_.strip())>2:
                    print(word_.split())


Overwriting code/twitter_clean_test_DV.py


In [4]:
%%sh  
python code/twitter_clean_test_DV.py > data/tweepy_cleaned_test.txt

bash.exe: warning: could not find /tmp, please create!


In [10]:
from pybloom import ScalableBloomFilter
sbf = ScalableBloomFilter(mode=ScalableBloomFilter.SMALL_SET_GROWTH)

final_dict={}
with open('data/tweepy_cleaned_train.txt') as file:
    for word in file:
        word_list=word.replace('[','').replace(']','').replace("'",'').split(',')
        for tweet_word in word_list:
            sbf.add(tweet_word)

with open('data/tweepy_cleaned_test.txt') as file:
    for word in file:
        word_list=word.replace('[','').replace(']','').replace("'",'').split(',')
        for tweet_word in word_list:
            if tweet_word in sbf:
                final_dict[tweet_word]=final_dict.get(tweet_word,0) + 1


print("(%s,%s)"%("cricket",final_dict['cricket']))
print("(%s,%s)"%("soccer",final_dict['soccer']))
print("(%s,%s)"%("basketball",final_dict['basketball']))
print("(%s,%s)"%("baseball",final_dict['baseball']))
print("(%s,%s)"%("tennis",final_dict['tennis']))

print('\n')

counter=0
print("################### TOP 10 WORDS ######################")
for key,value in sorted(final_dict.iteritems(),key=lambda (k,v):(v,k),reverse=True):
    counter+=1
    print("(%s,%s)"%(key,value))
    if counter>=10: break

(cricket,216)
(soccer,613)
(basketball,226)
(baseball,104)
(tennis,147)


################### TOP 10 WORDS ######################
(rt,16781)
( the,10354)
( to,8050)
( a,6313)
( soccer,5337)
( of,5289)
( basketball,5178)
( in,5154)
( is,4319)
( and,4025)


#### Tried using dynamic_pybloom (which worked)

In [2]:
from dynamic_pybloom import ScalableBloomFilter
sbf = ScalableBloomFilter(mode=ScalableBloomFilter.SMALL_SET_GROWTH)

final_dict={}
with open('data/tweepy_cleaned_train.txt') as file:
    for word in file:
        word_list=word.replace('[','').replace(']','').replace("'",'').split(',')
        for tweet_word in word_list:
            sbf.add(tweet_word)

with open('data/tweepy_cleaned_test.txt') as file:
    for word in file:
        word_list=word.replace('[','').replace(']','').replace("'",'').split(',')
        for tweet_word in word_list:
            if tweet_word in sbf:
                final_dict[tweet_word]=final_dict.get(tweet_word,0) + 1


print("(%s,%s)"%("cricket",final_dict['cricket']))
print("(%s,%s)"%("soccer",final_dict['soccer']))
print("(%s,%s)"%("basketball",final_dict['basketball']))
print("(%s,%s)"%("baseball",final_dict['baseball']))
print("(%s,%s)"%("tennis",final_dict['tennis']))

print('\n')



(cricket,216)
(soccer,613)
(basketball,226)
(baseball,104)
(tennis,147)


